In [95]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## Test On Static Image
Filler/Desc

In [12]:
input_img = cv2.imread('group_img_mask.jpg')
gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors =5)

for (x, y, w, h) in faces:
    cv2.rectangle(input_img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
cv2.imshow('image', input_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Test On Live 
Filler/Desc

In [3]:
class Basic_CNN(nn.Module):
    # 7 layers: 2 conv, 2 max pool, 2 linear, 1 output
    def __init__(self):
        self.name = "Basic_CNN"
        super(Basic_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5) #in_channels=3, out_chanels=5, kernel_size=5
        self.pool = nn.MaxPool2d(2, 2) #kernel_size=2, stride=2 
        self.conv2 = nn.Conv2d(5, 10, 5) #in_channels=5, out_chanels=10, kernel_size=5
        self.fc1 = nn.Linear(10*57*57, 32)
        self.fc2 = nn.Linear(32, 2)
    def weights_init(m):
        if isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform(m.weight.data)
            nn.init.xavier_uniform(m.bias.data)

    def forward(self, img):
        x = self.pool(F.relu(self.conv1(img))) #relu activation function
        x = self.pool(F.relu(self.conv2(x))) #relu activation function
        x = x.view(-1, 10*57*57)
        x = F.relu(self.fc1(x)) #relu activation function
        x = self.fc2(x)
        return x

In [22]:
model = Basic_CNN()
model_path = "models/model_{0}_bs{1}_lr{2}_epoch{3}".format("Basic_CNN", 32, 0.00014, 1)
state = torch.load(model_path)
model.load_state_dict(state)

<All keys matched successfully>

In [5]:
labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [96]:
source = cv2.VideoCapture(0) #Change parameter if you have different webcams, 0 for default

while(True):    
    
    #Read from webcam
    ret,img = source.read()
    
    #OpenCV convert to grayscale for haarcascade classifier
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors =5)
    
    #OpenCV convert to BGR to RGB for CNN
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    #Iterate through all faces detected
    for (x, y, w, h) in faces:
        
        #Boundaries for face
        rgb_face = rgb[y:y+w, x:x+w]
        
        #Resize & Normalize
        resized_face_img = cv2.resize(rgb_face, (240,240)) /255.0
        
        #Reshape tensor
        model_input = torch.tensor(resized_face_img).unsqueeze(0).permute(0,3,1,2) 
        
        #Model Prediction
        output = model(model_input.float())
        label = output.max(1, keepdim=True)[1].sum().item()
        
        #Draws boxes around faces
        cv2.rectangle(img, (x, y), (x + w, y + h), color_dict[label], 2)
        
    
    cv2.imshow('Detecting Masks... Press esc to exit',img)
    key=cv2.waitKey(1)
    
    #Exit - Esc key
    if(key==27):
        break
    
cv2.waitKey(0)
cv2.destroyAllWindows()